In [ ]:
import pytz  # Import pytz library for time zone handling
import cv2  # OpenCV which is used for image processing
from datetime import datetime  # Date and time handling
import sys  # System functions.
import requests  # API handling for getting weather data from OpenWeatherMap
import numpy as np  # Numerical operations
from PyQt6.QtWidgets import QApplication, QWidget, QLabel, QVBoxLayout, QLineEdit, QPushButton, QComboBox  # PyQt6 GUI components.
from PyQt6.QtGui import QFont  # GUI styling(fonts,...)
from PyQt6.QtCore import Qt  # Alignment options(positions)

# API Constants
API_KEY = "5197a49bf57f5916840fcf991dacd9c3"  # OpenWeather API Key for authentification
BASE_URL = "https://api.openweathermap.org/data/2.5/weather"  # endpoint for current weather data
FORECAST_URL = "https://api.openweathermap.org/data/2.5/forecast"  # endpoint for 5-day forecast data

# Languages for weather description.
LANGUAGES = {"English": "en", "French": "fr", "Spanish": "es", "Kinyarwanda": "rw"}

# Function to convert temperature between units
def convert_temperature(value, from_unit, to_unit):
    conversions = {
        ('C', 'F'): lambda x: (x * 9/5) + 32, # from celisius to fahreneit
        ('F', 'C'): lambda x: (x - 32) * 5/9,  # from fahrneit to celisius
        ('C', 'K'): lambda x: x + 273.15,      # celisius to kelvin
        ('K', 'C'): lambda x: x - 273.15,      # kelvin to celsius
        ('F', 'K'): lambda x: (x - 32) * 5/9 + 273.15,  # fahrneit to kelvin
        ('K', 'F'): lambda x: (x - 273.15) * 9/5 + 32,  # kelvin to fahrneit
    }
    return conversions.get((from_unit, to_unit), lambda x: x)(value)  # get the converted value 

# Function to download and process the background image
image_url = "https://ncas.ac.uk/app/uploads/2020/06/Eyjafjallajokull_takenbySDMobbs-1024x572.jpg"  # endpoint for background image
response = requests.get(image_url)  # download the image from the URL
if response.status_code == 200:  # confirm if the request was successful
    with open("background.jpg", "wb") as file:
        file.write(response.content)  # save image
    print("Image downloaded!")  # image saved
else:
    print("Failed to download image")  # when image is not found
    sys.exit(1)

# Apply blue tint to the background image
display_image = cv2.imread("background.jpg")  # Read image
if display_image is None:
    print("Error: Image not found.")  # message output when image is not found
    sys.exit(1)  # Exit

blue_tint = np.full_like(display_image, (255, 0, 0), dtype=np.uint8)  # Create blue tint overlay
blue_tinted = cv2.addWeighted(display_image, 0.7, blue_tint, 0.3, 0)  # Apply the blue tint
cv2.imwrite("background_tinted.jpg", blue_tinted)  # Save the tinted image

# Function to get current weather data and error handling
def get_weather(city, temp_unit="C", lang="en"): 
    params = {"q": city, "appid": API_KEY, "units": "metric", "lang": lang}  # API request dictionary for current weather
    try:
        response = requests.get(BASE_URL, params=params)  # API requesting data for current weather
        response.raise_for_status()  # Raise an error for bad response.
        data = response.json()  # Parse JSON format response
        temperature = data["main"]["temp"]  # Extracting temperature
        # Convert temperature if needed
        if temp_unit != "C":
            temperature = convert_temperature(temperature, "C", temp_unit)
        unit = "°C" if temp_unit == "C" else "°F" if temp_unit == "F" else "K"  # determining unit

        # Convert sunrise and sunset timestamps to local time (human-readable format)
        timezone_offset = data["timezone"]  # get offset in seconds (representing the difference UTC and local time)

        # Convert UTC time (timestamps) to local time using pytz library
        utc_time = datetime.utcfromtimestamp(data["sys"]["sunrise"])  # Convert sunrise timestamp to UTC datetime
        utc_time = pytz.utc.localize(utc_time)  # Localize the time as UTC
        local_timezone = pytz.timezone("Africa/Kigali")  #is to be changed to the intended local timezone 
        sunrise_time = utc_time.astimezone(local_timezone).strftime('%Y-%m-%d %H:%M:%S')  # Convert to local time and format it into human-readable format

        utc_time = datetime.utcfromtimestamp(data["sys"]["sunset"])  # Convert sunset timestamp to UTC datetime
        utc_time = pytz.utc.localize(utc_time)  # Localize the time as UTC
        sunset_time = utc_time.astimezone(local_timezone).strftime('%Y-%m-%d %H:%M:%S')  # Convert to local time and format it

        # Formatting string output
        return (
            f"City: {data['name']}\n"
            f"Temperature: {temperature:.2f}{unit}\n"
            f"Humidity: {data['main']['humidity']}%\n"
            f"Pressure: {data['main']['pressure']} hPa\n"
            f"Wind Speed: {data['wind']['speed']} m/s\n"
            f"Visibility: {data.get('visibility', 'N/A')} meters\n"
            f"Cloud Cover: {data['clouds']['all']}%\n"
            f"Sunrise: {sunrise_time}\n"
            f"Sunset: {sunset_time}\n"
            f"Description: {data['weather'][0]['description'].capitalize()}"
        )
    except requests.exceptions.HTTPError as err:
        if response.status_code == 404:
            return "Error: City not found."
    except Exception as err:
        return f"Unexpected Error: {str(err)}"

# Function to get 5-day weather forecast
def get_forecast(city, temp_unit="C", lang="en"):  #start fn that takes city, temp_unit and lang
    params = {"q": city, "appid": API_KEY, "units": "metric", "lang": lang}  # API request dictionary for forecast weather
    response = requests.get(FORECAST_URL, params=params)  # API requesting data for forecast
    if response.status_code == 200:
        data = response.json()  # Converts the response (which comes in JSON format) into a Python dictionary.
        forecast_text = "5-Day Forecast:\n"  # Initialize forecast text
        for i in range(0, len(data['list']), 8):  # Loop through forecast data (every 8 entries ~ 1 day)
            day_data = data['list'][i]  # Extract data for the day
            date = day_data['dt_txt'].split(" ")[0]  # Extract the date
            temp = day_data['main']['temp']  # Extracting temperature
            desc = day_data['weather'][0]['description'].capitalize()  # Extract weather description and capitalize first letter
            forecast_text += f"{date}: {temp:.2f}°C, {desc}\n"  # Format and append data
        return forecast_text
    else:
        return "Error: Forecast data not available."

# GUI Application class
class WeatherApp(QWidget):
    def __init__(self):
        super().__init__()
        self.initUI()  # Initializing the user interface

    def initUI(self):
        self.setWindowTitle("Weather App")  # setting the window title
        self.setGeometry(100, 100, 400, 700)  # setting the window size
        self.setStyleSheet("background-image: url('background_tinted.jpg'); background-size: cover;")  # setting the background image

        layout = QVBoxLayout()  # creating a vertical layout

        self.city_input = QLineEdit(self)  # field for city name input
        self.city_input.setPlaceholderText("Please enter city name")  # set the placeholder text
        self.city_input.setStyleSheet("color: white; background-color: rgba(0, 0, 0, 80); border: 1px solid white;")  # bright text on dark transparent background
        layout.addWidget(self.city_input)  # adding the input field to the layout

        self.unit_selector = QComboBox(self)  # adding dropdown for temperature unit selection
        self.unit_selector.addItems(["Celsius (°C)", "Fahrenheit (°F)", "Kelvin (K)"])  # adding the options to the dropdown
        self.unit_selector.setStyleSheet("color: white; background-color: rgba(0, 0, 0, 80); border: 1px solid white;")
        layout.addWidget(self.unit_selector)  # adding the dropdown to the layout

        self.language_selector = QComboBox(self)  # adding dropdown for language selection
        self.language_selector.addItems(LANGUAGES.keys())  # adding the options to the dropdown
        self.language_selector.setStyleSheet("color: white; background-color: rgba(0, 0, 0, 80); border: 1px solid white;")
        layout.addWidget(self.language_selector)

        self.get_weather_btn = QPushButton("Get Weather", self)  # creating a button
        self.get_weather_btn.clicked.connect(self.fetch_weather)  # connecting the button to the function that fetches weather data
        self.get_weather_btn.setStyleSheet("color: white; background-color: #005f99; border: 2px solid white; padding: 5px;")
        layout.addWidget(self.get_weather_btn)  # adding the button to the layout

        self.result_label = QLabel("Current weather", self)  # creating a label to display the weather result
        self.result_label.setFont(QFont("Times New Roman", 14))  # setting the font of the label
        self.result_label.setAlignment(Qt.AlignmentFlag.AlignCenter)  # setting the alignment of the label
        self.result_label.setStyleSheet("color: lightcyan; background-color: rgba(0, 0, 0, 100);")
        layout.addWidget(self.result_label)

        self.forecast_label = QLabel("5-Day Forecast", self)  # creating a label to display the forecast result
        self.forecast_label.setFont(QFont("Times New Roman", 14)) 
        self.forecast_label.setAlignment(Qt.AlignmentFlag.AlignCenter)
        self.forecast_label.setStyleSheet("color: lightcyan; background-color: rgba(0, 0, 0, 100);")
        layout.addWidget(self.forecast_label)

        self.setLayout(layout)

    def fetch_weather(self):
        city = self.city_input.text().strip()  # To user input and remove extra spaces
        unit = self.unit_selector.currentText()[0]  # To first letter of selected temperature unit (C, F, or K)
        lang = LANGUAGES[self.language_selector.currentText()]  # Get selected language code
        if city:
            weather_info = get_weather(city, unit, lang)  # Fetch the current weather data
            forecast_info = get_forecast(city, unit, lang)  # fetch the forecast data
            self.result_label.setText(weather_info)  # display the text"Display weather info" on the label
            self.forecast_label.setText(forecast_info)  # display the text"Display forecast info" on the label
        else:
            self.result_label.setText("Please enter a valid city name.")  # display the error message"Please enter a valid city name." on the label
            self.forecast_label.setText("")

if __name__ == "__main__":
    app = QApplication(sys.argv)
    window = WeatherApp()  # Creating application instance
    window.show()  # showing the window
    sys.exit(app.exec())  # Exiting the application when the window is closed


Image downloaded!


C:\Users\PC\AppData\Local\Temp\ipykernel_24900\3511877819.py:69: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  utc_time = datetime.utcfromtimestamp(data["sys"]["sunrise"])  # Convert sunrise timestamp to UTC datetime
C:\Users\PC\AppData\Local\Temp\ipykernel_24900\3511877819.py:74: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  utc_time = datetime.utcfromtimestamp(data["sys"]["sunset"])  # Convert sunset timestamp to UTC datetime


In [3]:
!pip install opencv-python


Defaulting to user installation because normal site-packages is not writeable
